https://archery.aqumon.com/sqlquery/

''''
sql code

SELECT
  id,uid, 
  case order_direction
     when 1 then "Buy"
     else "Sell"
     end as order_direction, 
  CASE order_type
    when 10 then "Market Order"
    else "Limit order"
    End as order_type,
  ticker, currency, finish_time, filled_price, filled_position, filled_amount,commission
FROM
  stock_trading.t_stock_task
WHERE
  create_time >= '2021-06-01' and filled_position != 0
  

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import os
import datetime as dt
import matplotlib.pyplot as plt
import pytz

%matplotlib inline
input_path = "/Users/ansonleung/Downloads"


all_files = glob.glob(os.path.join(input_path, 'export_result*'))
file_count = 0
for file in all_files:
    df = pd.read_excel(file, parse_dates=['finish_time'])
    file_count +=1
    
print('Possessed File: '+ str(file_count))

df.to_csv('/Users/ansonleung/Desktop/stock_trading_record.csv')





Possessed File: 1


In [7]:
df1 = df[df['currency']=='HKD'].drop(['id', 'uid','order_type','ticker','filled_price','filled_position'], axis =1)
df1['finish_time'] = df1['finish_time'].dt.date
df1a = df1[['finish_time','commission']]
df1a = df1a.groupby(['finish_time'],as_index=False).agg({'commission':'sum'})
df1 = pd.pivot_table(df1,index=["finish_time"],columns="order_direction",values=["filled_amount"],aggfunc=[np.sum,len])

df1.columns = df1.columns.get_level_values(2)
df1 = df1.set_axis(['HKD_Buy', 'HKD_Sell', 'HKD_Buy_Count','HKD_Sell_Count'], axis=1)
df1 = df1[['HKD_Buy_Count','HKD_Buy','HKD_Sell_Count','HKD_Sell']]
idx1= pd.date_range(min(df1.index), max(df1.index), freq='D')
df1 = df1.reindex(idx1, fill_value=0)
df2 = df[df['currency']=='USD']

df2['finish_time'] = df2['finish_time'].dt.tz_localize('Asia/Hong_Kong')
df2['finish_time'] = df2['finish_time'].dt.tz_convert('America/New_York')
df2['finish_time'] = df2['finish_time'].dt.date
df2['commission'] = df2['commission']*7.8
df2a = df2[['finish_time','commission']]
df2a = df2a.groupby(['finish_time'],as_index=False).agg({'commission':'sum'})

df2 = pd.pivot_table(df2,index=["finish_time"],columns="order_direction",values=["filled_amount"],aggfunc=[np.sum,len])

df2.columns = df2.columns.get_level_values(2)
df2 = df2.set_axis(['USD_Buy', 'USD_Sell', 'USD_Buy_Count','USD_Sell_Count'], axis=1)
df2 = df2[['USD_Buy_Count','USD_Buy','USD_Sell_Count','USD_Sell']]
idx2= pd.date_range(min(df2.index), max(df2.index), freq='D')
df2 = df2.reindex(idx2, fill_value=0)

df3 = df[df['currency']=='CNY'].drop(['id', 'uid','order_type','ticker','filled_price','filled_position'], axis =1)
df3['finish_time'] = df3['finish_time'].dt.date

df3a = df3[['finish_time','commission']]
df3a = df3a.groupby(['finish_time'],as_index=False).agg({'commission':'sum'})

df3 = pd.pivot_table(df3,index=["finish_time"],columns="order_direction",values=["filled_amount"],aggfunc=[np.sum,len])

df3.columns = df3.columns.get_level_values(2)
df3 = df3.set_axis(['CNY_Buy', 'CNY_Sell', 'CNY_Buy_Count','CNY_Sell_Count'], axis=1)
df3 = df3[['CNY_Buy_Count','CNY_Buy','CNY_Sell_Count','CNY_Sell']]
idx3= pd.date_range(min(df3.index), max(df3.index), freq='D')
df3 = df3.reindex(idx3, fill_value=0)


merged_df = pd.concat([df1, df2,df3],axis=1)
newdf= pd.merge(df1a,df2a,on='finish_time',how='outer')
newdf = pd.merge(newdf,df3a,on='finish_time',how='outer')
newdf = newdf.sort_values(by=['finish_time'])
newdf["total_commission"] = newdf.sum(axis=1)
newdf=newdf.set_index('finish_time')
merged_df = merged_df.join(newdf)
merged_df= merged_df.rename(columns={'commission_x': 'HK_commission', 'commission_y': 'US_commission','commission': 'CN_commission'}).round(0).fillna(0)
merged_df.to_csv('/Users/ansonleung/Desktop/stock_trading_summary.csv')

<ipython-input-7-ee8b01c9a97e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['finish_time'] = df2['finish_time'].dt.tz_localize('Asia/Hong_Kong')
<ipython-input-7-ee8b01c9a97e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['finish_time'] = df2['finish_time'].dt.tz_convert('America/New_York')
<ipython-input-7-ee8b01c9a97e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [8]:
merged_df.tail(4)

,HKD_Buy_Count,HKD_Buy,HKD_Sell_Count,HKD_Sell,USD_Buy_Count,USD_Buy,USD_Sell_Count,USD_Sell,CNY_Buy_Count,CNY_Buy,CNY_Sell_Count,CNY_Sell,HK_commission,US_commission,CN_commission,total_commission
2021-08-24,15.0,371378.0,12.0,197208.0,17.0,15719.0,7.0,5759.0,0.0,0.0,0.0,0.0,1510.0,346.0,0.0,1857.0
2021-08-25,18.0,203354.0,21.0,53668.0,18.0,11601.0,9.0,7806.0,0.0,0.0,0.0,0.0,1176.0,369.0,0.0,1545.0
2021-08-26,33.0,82452.0,14.0,2276.0,32.0,26093.0,6.0,8102.0,0.0,0.0,0.0,0.0,1296.0,575.0,0.0,1872.0
2021-08-27,29.0,137398.0,11.0,406214.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1785.0,0.0,0.0,1785.0


In [5]:
import pandas as pd
import csv
import glob
import os
import sys
import numpy as np
index = 0
input_path = "/Users/ansonleung/Downloads"
#all_files = glob.glob(os.path.join(input_path, 'Yearning_Data*'))
all_files = glob.glob(os.path.join(input_path, 'export_result*'))
for i in all_files: 
    if os.path.exists(i):
        os.remove(i)
        index+=1
print(f'{index} files deleted')

2 files deleted
